## 8 Puzzle 업데이트
- 탐색 문제 풀이 적합하게 관련된 속성과 메소드 추가
  - 깊이 정보 ```self.depth```
  - 목표 상태 ```self.goal```
  - 경로 비용 ```self.cost()```

In [23]:
# 클래스 설계 (상태, 전이모형, 그외 함수 포함)
# 깊이 정보와 목표 상태 및 경로 비용 함수 추가

class Puzzle:
  def __init__(self, board, depth=0):
    self.board = board # 현재 상태
    self.depth = depth # 현재 깊이
    

  def get_new_board(self, i1, i2, depth):
    new_board = self.board[:]
    new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
    return Puzzle(new_board, depth) # 목표 및 깊이 정보 추가 & 경로 비용 업데이트 

 
  def expand(self, depth):
    result = []
    i = self.board.index(0)	
    if not i in [0, 3, 6] :		# LEFT
      result.append(self.get_new_board(i, i-1, depth))
    if not i in [0, 1, 2] :		# UP  
      result.append(self.get_new_board(i, i-3, depth))
    if not i in [2, 5, 8]:		# RIGHT 
      result.append(self.get_new_board(i, i+1, depth))
    if not i in [6, 7, 8]:		# DOWN
      result.append(self.get_new_board(i, i+3, depth))
    return result

  def cost(self):
    return self.depth

  def __str__(self):
    return str(self.board[:3]) +"\n"+\
    str(self.board[3:6]) +"\n"+\
    str(self.board[6:]) +"\n"+\
    "------------------"

  def __eq__(self, other):
    return self.board == other.board

  def __ne__(self, other):
    return self.board != other.board
  
  # 우선순위 큐 사용 시 값의 비교가 발생함.
  # 이 때문에 less/greater than 연산을 사전 정의해야 오류 발생 안함
  def __lt__(self, other):
    return self.cost() < other.cost()

  def __gt__(self, other):
    return self.cost() > other.cost()

In [2]:
start = [2, 8, 3, 
         1, 6, 4, 
         7, 0, 5]

goal = [8, 6, 3, 
        2, 0, 4, 
        1, 7, 5]

### 균일 비용 알고리즘 위한 퍼즐 업데이트 버전 (PuzzleC)
- 경로비용 ```self.path_cost```
- ```expand()``` 함수 내 경로 비용 업데이트(경로 비용 누적)
    - 위, 아래 이동 시 3
    - 왼쪽, 오른쪽 이동 시 1

In [25]:
class PuzzleC:
    def __init__(self, board, depth=0, cost=0):
        self.board = board
        self.depth = depth
        self.path_cost = cost

    def get_new_board(self, i1, i2, depth, cost):
        new_board = self.board[:]
        new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
        return PuzzleC(new_board, depth, cost)
    
    def expand(self, depth):
        result = []
        i = self.board.index(0)	
        if not i in [0, 3, 6] :		# LEFT
            new_cost = self.path_cost + 1
            result.append(self.get_new_board(i, i-1, depth, new_cost))
        if not i in [0, 1, 2] :		# UP  
            new_cost = self.path_cost + 3
            result.append(self.get_new_board(i, i-3, depth, new_cost))
        if not i in [2, 5, 8]:		# RIGHT 
            new_cost = self.path_cost + 1
            result.append(self.get_new_board(i, i+1, depth, new_cost))
        if not i in [6, 7, 8]:		# DOWN
            new_cost = self.path_cost + 3
            result.append(self.get_new_board(i, i+3, depth, new_cost))
        return result

    def cost(self):
        return self.depth

    def __str__(self):
        return str(self.board[:3]) +"\n"+\
            str(self.board[3:6]) +"\n"+\
            str(self.board[6:]) +"\n"+\
            "------------------"

    def __eq__(self, other):
        return self.board == other.board

    def __ne__(self, other):
        return self.board != other.board
  
  # 우선순위 큐 사용 시 값의 비교가 발생함.
  # 이 때문에 less/greater than 연산을 사전 정의해야 오류 발생 안함
    def __lt__(self, other):
        return self.cost() < other.cost()

    def __gt__(self, other):
        return self.cost() > other.cost()
        

## Uninformed Search

### 너비우선탐색(Breath-First Search)
- 루트 노드의 모든 자식 노드들을 탐색한 후에 해가 발견되지 않으면 한 레벨 내려가서 동일한 방법으로 탐색을 계속하는 방법
- FIFO 대기열 활용 (큐)


### 의사코드
```java
Function BFS(initial_state, goal_state)
  queue ← [initial_state]
  visited ← [ ]
  while queue != [ ] do
    current_state ← queue의 첫번째 요소
    if current_state == goal_state
      return SUCCESS
    else
      current_state를 visited에 추가
      current_state의 자식 노드를 생성
      if current_state의 자식 노드가 이미 queue나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        남은 자식 노드들은 queue의 마지막에 추가
  return FAIL
```

In [26]:
from collections import deque

def run_bfs(initial_state, goal_state):
    # 실습
    queue = deque()
    visited = []
    queue.append(initial_state)

    count = 1

    while queue:
        current_state = queue.popleft()  #가장 처음에 들어간 왼쪽부터 꺼내기
        print(f"count:{count}, depth:{current_state.depth}\n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "success"
        
        depth = current_state.depth + 1
        visited.append(current_state)

        for state in current_state.expand(depth):
            if (state in visited) or (state in queue):
                continue
            else:
                queue.append(state)
    return "failed"

initial_state = Puzzle(start)
answer = run_bfs(initial_state, goal)
print(answer)

count:1, depth:0
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
count:2, depth:1
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
count:3, depth:1
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]
------------------
count:4, depth:1
[2, 8, 3]
[1, 6, 4]
[7, 5, 0]
------------------
count:5, depth:2
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
count:6, depth:2
[2, 8, 3]
[0, 1, 4]
[7, 6, 5]
------------------
count:7, depth:2
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------------------
count:8, depth:2
[2, 8, 3]
[1, 4, 0]
[7, 6, 5]
------------------
count:9, depth:2
[2, 8, 3]
[1, 6, 0]
[7, 5, 4]
------------------
count:10, depth:3
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]
------------------
count:11, depth:3
[2, 8, 3]
[6, 0, 4]
[1, 7, 5]
------------------
count:12, depth:3
[0, 8, 3]
[2, 1, 4]
[7, 6, 5]
------------------
count:13, depth:3
[2, 8, 3]
[7, 1, 4]
[0, 6, 5]
------------------
count:14, depth:3
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]
------------------
count:15, depth:3
[2, 3, 0]
[1, 8, 4]
[7, 6, 5]
------------------
coun

### 균일 비용 탐색(Uniform-Cost Search)
- 가장 적은 비용으로 목표 상태에 도달하는 경로를 찾는 데 사용하는 방법
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (경로비용, 상태)로 표현

### 의사코드
```java
Function UCS(initial_state, goal_state)
  p_queue ← [(cost, initial_state)]
  visited ← [ ] 
  while p_queue != [ ] do
    current_cost, current_state ← p_queue 에서 경로 비용이 가장 낮은 상태
    if current_state == goal_state
      return SUCCESS
    else
      current_state를 visited에 추가 
      current_state의 자식 노드 생성
      if current_state의 자식 노드가 p_queue 이나 visited 에 있으면
        해당 자식 노드 건너뜀
      else
        자식 노드의 경로 비용을 계산 
        자식 노드들을 p_queue에 추가
return FAIL
```

In [27]:
import heapq

def run_ucs(initial_state, goal_state):
    # 실습
    pqueue = []
    visited = []
    heapq.heappush(pqueue, (initial_state.cost(), initial_state))

    count =1
    while pqueue:
        current_cost, current_state = heapq.heappop(pqueue)
        print(f"count:{count}, depth:{current_state.depth} \n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "success"
        
        depth = current_state.depth + 1
        visited.append(current_state)
        for state in current_state.expand(depth):
            if (state in visited) or (state in [s[1] for s in pqueue]):
                continue
            else:
                heapq.heappush(pqueue, (state.cost(), state))

# 초기 상태와 목표 상태는 Puzzle 클래스로 정의된다고 가정
initial_state = PuzzleC(start)
answer = run_ucs(initial_state, goal)
print(answer)

count:1, depth:0 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
count:2, depth:1 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
count:3, depth:1 
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]
------------------
count:4, depth:1 
[2, 8, 3]
[1, 6, 4]
[7, 5, 0]
------------------
count:5, depth:2 
[2, 8, 3]
[0, 1, 4]
[7, 6, 5]
------------------
count:6, depth:2 
[2, 8, 3]
[1, 4, 0]
[7, 6, 5]
------------------
count:7, depth:2 
[2, 8, 3]
[1, 6, 0]
[7, 5, 4]
------------------
count:8, depth:2 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
count:9, depth:2 
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------------------
count:10, depth:3 
[2, 8, 3]
[7, 1, 4]
[0, 6, 5]
------------------
count:11, depth:3 
[2, 8, 3]
[1, 0, 6]
[7, 5, 4]
------------------
count:12, depth:3 
[2, 8, 3]
[6, 0, 4]
[1, 7, 5]
------------------
count:13, depth:3 
[2, 3, 0]
[1, 8, 4]
[7, 6, 5]
------------------
count:14, depth:3 
[2, 8, 0]
[1, 4, 3]
[7, 6, 5]
------------------
count:15, depth:3 
[0, 8, 3]
[2, 1, 4]
[7, 6, 5]
--------

### 깊이우선탐색(Depth-First Search) / 깊이 제한 탐색 (Depth-Limited Search)
- 항상 탐색 트리의 전선에서 가장 깊은 수준, 즉 노드들이 더이상 자식 노드가 없는 수준까지 나아간 후, 이전 노드로 후퇴하며 탐색하는 기법
- (본 실습에서는 탐색 효율을 고려하여 깊이 제한 방식으로 구현)
- LIFO 대기열 활용 (스택)

### 의사코드
```java
Function DLS(initial_state, goal_state)
  stack ← [initial_state]
  visited ← [ ]
  while stack != [ ] do
    current_state ← stack의 첫번째 요소
    if current_state == goal_state
      return SUCCESS
    if 현재 깊이 > 깊이 제한
      다음 반복으로 건너뛰기
    else
      current_state를 visited에 추가
      current_state의 자식 노드 생성
      if current_state의 자식 노드가 이미 stack이나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        남은 자식 노드들은 stack의 처음에 추가
  return FAIL
```

In [28]:
from collections import deque

def run_dfs(initial_state, goal_state):
    # 실습
    stack = deque()
    visited = [ ]

    stack.append(initial_state)
    count = 1

    while stack:
        current_state = stack.pop()
        print(f"count:{count}, depth:{current_state.depth}\n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "success"
        
        depth = current_state.depth + 1
        visited.append(current_state)

        if depth > 5:
            continue

        for state in reversed(current_state.expand(depth)):
            if(state in visited) or (state in stack):
                continue
            else:
                stack.append(state)

    return "failed"

initial_state = Puzzle(start)
answer=run_dfs(initial_state, goal)
print(answer)

count:1, depth:0
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
count:2, depth:1
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
count:3, depth:2
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
count:4, depth:3
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]
------------------
count:5, depth:4
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]
------------------
count:6, depth:5
[8, 3, 0]
[2, 6, 4]
[1, 7, 5]
------------------
count:7, depth:5
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]
------------------
success


### 

# Informed Search

### 8 Puzzle + 휴리스틱 
- 정보 있는 탐색을 위해 휴리스틱 함수와 평가 함수를 추가
- 평가함수 ```self.f()```
- 휴리스틱 ```self.h()```
- 경로비용 ```self.g()```

In [4]:
class PuzzleH:
  def __init__(self, board, goal, depth=0):
    self.board = board
    self.depth = depth
    self.goal = goal

  def get_new_board(self, i1, i2, depth):
    new_board = self.board[:]
    new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
    return PuzzleH(new_board, self.goal, depth)

  def expand(self, depth):
    result = []
    i = self.board.index(0)
    if not i in [0, 3, 6]:
      result.append(self.get_new_board(i, i-1, depth))
    if not i in [0, 1, 2]:
      result.append(self.get_new_board(i, i-3, depth))
    if not i in [2, 5, 8]:	
      result.append(self.get_new_board(i, i+1, depth))
    if not i in [6, 7, 8]:
      result.append(self.get_new_board(i, i+3, depth))
    return result


  def f(self):
    return self.h()+self.g()

  # 휴리스틱 함수 h(n): 현재 제 위치에 있지 않은 타일의 개수를 계산하여 반환
  def h(self):
    score = 0
    for i in range(9):
      if (self.board[i] != 0) and (self.board[i] != self.goal[i]):
        score += 1
    return score

  # 경로 비용 함수 g(n): 시작 노드로부터의 깊이를 반환
  def g(self):
    return self.depth

  def __eq__(self, other):
    return self.board == other.board

  def __ne__(self, other):
    return self.board != other.board

  def __lt__(self, other):
    return self.f() < other.f()

  def __gt__(self, other):
    return self.f() > other.f()

  def __str__(self):
    return str(self.board[:3]) +"\n"+\
    str(self.board[3:6]) +"\n"+\
    str(self.board[6:]) +"\n"

### 최우선탐색 (Greedy Best-First Search)
- 현재 상태에서 가장 최선이라고 생각되는 것을 선택하는 방식으로 동작하는 방법, 즉, 휴리스틱 값이 작은 노드를 우선적으로 탐색하는 방법
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (휴리스틱 값, 상태)로 표현

### 의사코드
```java
Function GBFS(initial_state, goal_state)
  p_queue ← [(h, initial_state)]
  visited ← [ ]
  while p_queue != [ ] do
    current_h, current_state ← p_queue에서 휴리스틱 h값이 가장 작은 노드 추출
    if current_state == goal_state
      return SUCCESS
    else
      current_state의 자식 노드 생성
      current_state를 visited에 추가
      if current_state의 자식 노드가 이미 p_queue나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        자식 노드의 휴리스틱 값 h(n)을 계산
        남은 자식 노드들은 h값 기준으로 p_queue에 추가
  return FAIL
```

In [9]:
import heapq

def run_gbfs(initial_state, goal_state):
    # 실습
    pqueue = []
    visited = []
    heapq.heappush(pqueue, (initial_state.h(), initial_state))

    count =1
    while pqueue:
        current_h, current_state = heapq.heappop(pqueue)
        print(f"count:{count}, h:{current_state.h()} \n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "success"
        
        depth = current_state.depth + 1
        visited.append(current_state)
        for state in current_state.expand(depth):
            if (state in visited) and (state in [s[1] for s in pqueue]):
                continue
            else:
                heapq.heappush(pqueue, (state.h(), state))
    return "failed"

initial_state = PuzzleH(start, goal)
run_gbfs(initial_state, goal)

count:1, h:5 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

count:2, h:4 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

count:3, h:3 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]

count:4, h:2 
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]

count:5, h:1 
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]

count:6, h:0 
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]



'success'

## A* 알고리즘
- 초기 노드에서 목표 노드까지의 최단 경로를 찾는 효율적인 알고리즘으로, f(n) = h(n) + g(n) 활용해 평가 함수로 활용
  1) 시작 노드에서 현재 노드까지의 경로 비용 g(n)
  2) 휴리스틱을 사용하여 현재 노드에서 목표 노드까지 이동하는 데 드는 예상 비용 h(n)
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (f(n), 상태)로 표현

### 의사코드
```java
Function AStar(initial_state, goal_state)
  p_queue ← [f, initial_state]
  visited ← [ ]
  while p_queue != [ ] do
    current_f, current_state ← p_queue 에서 가장 평가 함수 f(n) 값이 좋은 노드
    if current_state == goal_state
      return SUCCESS
    else
      current_state의 자식 노드 생성
      current_state를 visited에 추가
      if current_state의 자식 노드가 p_queue 이나 visited 에 있으면
        해당 자식 노드 건너뜀
      else
        자식 노드의 평가 함수 값 f(n) = g(n) + h(n)을 계산
        자식 노드들을 f 값 기준으로 p_queue 에 추가
  return FAIL
```

In [8]:
import heapq

def run_astar(initial_state, goal_state):
    # 실습
    pqueue = []
    visited = []
    heapq.heappush(pqueue, (initial_state.f(), initial_state))

    count =1
    while pqueue:
        current_f, current_state = heapq.heappop(pqueue)
        print(f"count:{count}, f:{current_state.f()}, h:{current_state.h()}+g:{current_state.g()} \n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "success"
        
        depth = current_state.depth + 1
        visited.append(current_state)
        for state in current_state.expand(depth):
            if (state in visited) and (state in [s[1] for s in pqueue]):
                continue
            else:
                heapq.heappush(pqueue, (state.f(), state))
    return "failed"

# 초기 상태와 목표 상태는 Puzzle 클래스로 정의된다고 가정
initial_state = PuzzleH(start, goal)
run_astar(initial_state, goal)

count:1, f:5, h:5+g:0 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]

count:2, f:5, h:4+g:1 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]

count:3, f:5, h:3+g:2 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]

count:4, f:5, h:2+g:3 
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]

count:5, f:5, h:1+g:4 
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]

count:6, f:5, h:0+g:5 
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]



'success'